In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
# import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from wordcloud import WordCloud
import re
import os
from sqlalchemy import create_engine # database connection
import datetime as dt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from datetime import datetime
from bs4 import BeautifulSoup
from sklearn.model_selection import GridSearchCV


In [3]:
df = pd.read_csv(r'F:\mpst_full_data.csv', delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 14828 rows and 6 columns


In [4]:
df.head(5)

,imdb_id,title,plot_synopsis,tags,split,synopsis_source
0,tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
1,tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
2,tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
3,tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
4,tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras import preprocessing

Using TensorFlow backend.


In [6]:
# df=df.drop(columns=["split"])

In [7]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split',
       'synopsis_source'],
      dtype='object')

In [8]:
# df['combined']=df['title']+". "+ df['plot_synopsis']

In [9]:
type(Sequential()) == Sequential


True

In [10]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.sequence import pad_sequences

from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
# import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot

In [11]:
def gdd(x):
    a=x.split()
    return len(a)

In [12]:
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [14]:
from tqdm import tqdm
preprocessed_synopsis = []
# tqdm is for printing the status bar
for sentance in tqdm(df['plot_synopsis'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_synopsis.append(sentance.strip())

100%|███████████████████████████████████████████████████████████████████████████| 14828/14828 [00:34<00:00, 430.55it/s]


In [15]:
df['preprocessed_plots']=preprocessed_synopsis

In [16]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [17]:
def remove_spaces(x):
    x=x.split(",")
    nospace=[]
    for item in x:
        item=item.lstrip()
        nospace.append(item)
    return (",").join(nospace)  

In [18]:
df['tags']=df['tags'].apply(remove_spaces)

In [19]:
train=df.loc[df.split=='train']
cv=df.loc[df.split=="val"]
cv=cv.reset_index()
train=train.reset_index()
test=df.loc[df.split=='test']
test=test.reset_index()

In [20]:
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
y_train = vectorizer.fit_transform(train['tags']).toarray()
y_test=vectorizer.transform(test['tags']).toarray()
y_cv=vectorizer.transform(cv['tags']).toarray()

In [21]:
import tensorflow as tf

session_conf = tf.ConfigProto(
 intra_op_parallelism_threads=1, 
                        inter_op_parallelism_threads=1, 
                      )

In [22]:
df.columns

Index(['imdb_id', 'title', 'plot_synopsis', 'tags', 'split', 'synopsis_source',
       'preprocessed_plots'],
      dtype='object')

In [23]:
max(df['preprocessed_plots'].apply(gdd))

6044

In [24]:
vect=Tokenizer()
vect.fit_on_texts(train['preprocessed_plots'])
vocab_size = len(vect.word_index) + 1

In [25]:
vocab_size

98782

In [26]:
encoded_docs_train = vect.texts_to_sequences(train['preprocessed_plots'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1200, padding='post')
print(padded_docs_train)

[[  680  4577 54887 ...     0     0     0]
 [    6  4356    69 ...     0     0     0]
 [ 2953   323   108 ...    19    66  6797]
 ...
 [   66  2554   401 ...     0     0     0]
 [ 4799  2605  2908 ...     0     0     0]
 [ 1160  2299  2425 ...     0     0     0]]


In [27]:
encoded_docs_test = vect.texts_to_sequences(test['preprocessed_plots'])
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1200, padding='post')
encoded_docs_cv = vect.texts_to_sequences(cv['preprocessed_plots'])
padded_docs_cv = pad_sequences(encoded_docs_cv, maxlen=1200, padding='post')



In [28]:
def f1micro(y_true, y_pred):
    return tf.py_func(f1_score(y_true, y_pred,average='mirco'),tf.double)


In [29]:
# embedding_matrix.shape

In [30]:
# embedding_matrix.shape[1]

In [ ]:
# timesteps = len(x_train[0])
# input_dim = len(x_train[0])
# # n_classes = _count_classes(y_train)
# n_classes=147
# print(timesteps)
# print(input_dim)
# print(len(x_train[0]))

In [ ]:
# model_glove = Sequential()
# model_glove.add(Embedding(vocab_size,output_dim=embedding_matrix.shape[1],input_length=1200,weights=[embedding_matrix]))
# model_glove.add(LSTM(256, return_sequences=True,activation='relu'))
# model_glove.add(Flatten())
# model_glove.add(Dense(256, activation='relu'))
# model_glove.add(Dropout(0.5))
# model_glove.add(Dense(256, activation='relu'))
# model_glove.add(Dropout(0.5))
# model_glove.add(Dense(71, activation='sigmoid'))
# model_glove.compile(loss='binary_crossentropy', optimizer='adam')
# ## Fit train data
# model_glove.fit(padded_docs, y_train,validation_data=(padded_docs_cv,y_cv), epochs =5,batch_size=32)

In [67]:
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from time import time
from keras.callbacks import TensorBoard
from keras.layers.embeddings import Embedding
from keras.callbacks import ReduceLROnPlateau

In [68]:
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_f1micro', verbose=0, save_best_only=True, mode='max')

In [69]:
model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_11 (Dropout)         (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 71)                3621      
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________


In [70]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-simple1.hdf5', save_best_only=True)
]

history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)
# history=model.fit(padded_docs_train, y_train,validation_data=(padded_docs_cv,y_cv), epochs =5,batch_size=32,callbacks=[tensorboard])

Train on 8540 samples, validate on 949 samples
Epoch 1/5


4640/8540 [===============>..............] - ETA: 12:25 - loss: 0.6903 - categorical_accuracy: 0.0000e+0 - ETA: 6:45 - loss: 0.6895 - categorical_accuracy: 0.0000e+0 - ETA: 4:54 - loss: 0.6886 - categorical_accuracy: 0.0000e+ - ETA: 3:57 - loss: 0.6877 - categorical_accuracy: 0.0000e+ - ETA: 3:23 - loss: 0.6867 - categorical_accuracy: 0.0000e+ - ETA: 3:00 - loss: 0.6857 - categorical_accuracy: 0.0052   - ETA: 2:43 - loss: 0.6848 - categorical_accuracy: 0.00 - ETA: 2:31 - loss: 0.6838 - categorical_accuracy: 0.00 - ETA: 2:21 - loss: 0.6827 - categorical_accuracy: 0.00 - ETA: 2:13 - loss: 0.6817 - categorical_accuracy: 0.00 - ETA: 2:07 - loss: 0.6807 - categorical_accuracy: 0.00 - ETA: 2:02 - loss: 0.6797 - categorical_accuracy: 0.00 - ETA: 1:57 - loss: 0.6787 - categorical_accuracy: 0.00 - ETA: 1:54 - loss: 0.6777 - categorical_accuracy: 0.00 - ETA: 1:50 - loss: 0.6767 - categorical_accuracy: 0.00 - ETA: 1:47 - loss: 0.6756 - categorical_accuracy: 0.00 - ETA: 1:45 - loss: 0.6746 - categ

8540/8540 [==============================] - ETA: 35s - loss: 0.4626 - categorical_accuracy: 0.101 - ETA: 35s - loss: 0.4610 - categorical_accuracy: 0.102 - ETA: 34s - loss: 0.4593 - categorical_accuracy: 0.101 - ETA: 34s - loss: 0.4579 - categorical_accuracy: 0.101 - ETA: 34s - loss: 0.4563 - categorical_accuracy: 0.101 - ETA: 34s - loss: 0.4548 - categorical_accuracy: 0.101 - ETA: 33s - loss: 0.4532 - categorical_accuracy: 0.102 - ETA: 33s - loss: 0.4516 - categorical_accuracy: 0.102 - ETA: 33s - loss: 0.4500 - categorical_accuracy: 0.102 - ETA: 32s - loss: 0.4485 - categorical_accuracy: 0.103 - ETA: 32s - loss: 0.4469 - categorical_accuracy: 0.103 - ETA: 32s - loss: 0.4455 - categorical_accuracy: 0.103 - ETA: 31s - loss: 0.4439 - categorical_accuracy: 0.104 - ETA: 31s - loss: 0.4425 - categorical_accuracy: 0.104 - ETA: 31s - loss: 0.4411 - categorical_accuracy: 0.104 - ETA: 31s - loss: 0.4397 - categorical_accuracy: 0.104 - ETA: 30s - loss: 0.4381 - categorical_accuracy: 0.104 - ETA

4672/8540 [===============>..............] - ETA: 52s - loss: 0.1491 - categorical_accuracy: 0.250 - ETA: 54s - loss: 0.1398 - categorical_accuracy: 0.218 - ETA: 53s - loss: 0.1377 - categorical_accuracy: 0.197 - ETA: 54s - loss: 0.1369 - categorical_accuracy: 0.195 - ETA: 54s - loss: 0.1382 - categorical_accuracy: 0.181 - ETA: 54s - loss: 0.1404 - categorical_accuracy: 0.192 - ETA: 54s - loss: 0.1434 - categorical_accuracy: 0.187 - ETA: 54s - loss: 0.1464 - categorical_accuracy: 0.168 - ETA: 53s - loss: 0.1441 - categorical_accuracy: 0.173 - ETA: 53s - loss: 0.1464 - categorical_accuracy: 0.168 - ETA: 52s - loss: 0.1469 - categorical_accuracy: 0.164 - ETA: 52s - loss: 0.1481 - categorical_accuracy: 0.156 - ETA: 52s - loss: 0.1491 - categorical_accuracy: 0.153 - ETA: 51s - loss: 0.1496 - categorical_accuracy: 0.151 - ETA: 51s - loss: 0.1478 - categorical_accuracy: 0.158 - ETA: 51s - loss: 0.1470 - categorical_accuracy: 0.152 - ETA: 50s - loss: 0.1476 - categorical_accuracy: 0.154 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.1462 - categorical_accuracy: 0.144 - ETA: 23s - loss: 0.1462 - categorical_accuracy: 0.144 - ETA: 23s - loss: 0.1463 - categorical_accuracy: 0.144 - ETA: 23s - loss: 0.1464 - categorical_accuracy: 0.144 - ETA: 23s - loss: 0.1464 - categorical_accuracy: 0.145 - ETA: 22s - loss: 0.1461 - categorical_accuracy: 0.146 - ETA: 22s - loss: 0.1463 - categorical_accuracy: 0.145 - ETA: 22s - loss: 0.1462 - categorical_accuracy: 0.145 - ETA: 22s - loss: 0.1460 - categorical_accuracy: 0.146 - ETA: 22s - loss: 0.1459 - categorical_accuracy: 0.146 - ETA: 21s - loss: 0.1458 - categorical_accuracy: 0.146 - ETA: 21s - loss: 0.1459 - categorical_accuracy: 0.146 - ETA: 21s - loss: 0.1459 - categorical_accuracy: 0.146 - ETA: 21s - loss: 0.1458 - categorical_accuracy: 0.147 - ETA: 21s - loss: 0.1458 - categorical_accuracy: 0.147 - ETA: 20s - loss: 0.1458 - categorical_accuracy: 0.146 - ETA: 20s - loss: 0.1458 - categorical_accuracy: 0.146 - ETA

...

8540/8540 [==============================] - ETA: 23s - loss: 0.1434 - categorical_accuracy: 0.140 - ETA: 23s - loss: 0.1433 - categorical_accuracy: 0.140 - ETA: 23s - loss: 0.1433 - categorical_accuracy: 0.140 - ETA: 23s - loss: 0.1433 - categorical_accuracy: 0.141 - ETA: 23s - loss: 0.1432 - categorical_accuracy: 0.140 - ETA: 22s - loss: 0.1430 - categorical_accuracy: 0.141 - ETA: 22s - loss: 0.1429 - categorical_accuracy: 0.140 - ETA: 22s - loss: 0.1431 - categorical_accuracy: 0.140 - ETA: 22s - loss: 0.1430 - categorical_accuracy: 0.141 - ETA: 22s - loss: 0.1432 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1431 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1431 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1429 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1428 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1430 - categorical_accuracy: 0.140 - ETA: 21s - loss: 0.1430 - categorical_accuracy: 0.141 - ETA: 20s - loss: 0.1428 - categorical_accuracy: 0.141 - ETA

4672/8540 [===============>..............] - ETA: 53s - loss: 0.1647 - categorical_accuracy: 0.125 - ETA: 53s - loss: 0.1595 - categorical_accuracy: 0.125 - ETA: 52s - loss: 0.1662 - categorical_accuracy: 0.135 - ETA: 52s - loss: 0.1607 - categorical_accuracy: 0.117 - ETA: 52s - loss: 0.1573 - categorical_accuracy: 0.125 - ETA: 52s - loss: 0.1540 - categorical_accuracy: 0.114 - ETA: 51s - loss: 0.1494 - categorical_accuracy: 0.138 - ETA: 51s - loss: 0.1493 - categorical_accuracy: 0.132 - ETA: 51s - loss: 0.1470 - categorical_accuracy: 0.135 - ETA: 51s - loss: 0.1486 - categorical_accuracy: 0.131 - ETA: 50s - loss: 0.1491 - categorical_accuracy: 0.133 - ETA: 50s - loss: 0.1492 - categorical_accuracy: 0.132 - ETA: 50s - loss: 0.1487 - categorical_accuracy: 0.137 - ETA: 50s - loss: 0.1495 - categorical_accuracy: 0.133 - ETA: 49s - loss: 0.1481 - categorical_accuracy: 0.139 - ETA: 49s - loss: 0.1458 - categorical_accuracy: 0.140 - ETA: 49s - loss: 0.1451 - categorical_accuracy: 0.141 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.1429 - categorical_accuracy: 0.138 - ETA: 23s - loss: 0.1429 - categorical_accuracy: 0.138 - ETA: 23s - loss: 0.1429 - categorical_accuracy: 0.138 - ETA: 22s - loss: 0.1430 - categorical_accuracy: 0.138 - ETA: 22s - loss: 0.1429 - categorical_accuracy: 0.138 - ETA: 22s - loss: 0.1431 - categorical_accuracy: 0.138 - ETA: 22s - loss: 0.1431 - categorical_accuracy: 0.139 - ETA: 22s - loss: 0.1429 - categorical_accuracy: 0.140 - ETA: 21s - loss: 0.1431 - categorical_accuracy: 0.140 - ETA: 21s - loss: 0.1430 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1428 - categorical_accuracy: 0.141 - ETA: 21s - loss: 0.1429 - categorical_accuracy: 0.140 - ETA: 21s - loss: 0.1428 - categorical_accuracy: 0.140 - ETA: 20s - loss: 0.1426 - categorical_accuracy: 0.140 - ETA: 20s - loss: 0.1426 - categorical_accuracy: 0.140 - ETA: 20s - loss: 0.1427 - categorical_accuracy: 0.141 - ETA: 20s - loss: 0.1426 - categorical_accuracy: 0.141 - ETA

4672/8540 [===============>..............] - ETA: 54s - loss: 0.1349 - categorical_accuracy: 0.250 - ETA: 52s - loss: 0.1298 - categorical_accuracy: 0.203 - ETA: 52s - loss: 0.1307 - categorical_accuracy: 0.156 - ETA: 52s - loss: 0.1247 - categorical_accuracy: 0.148 - ETA: 52s - loss: 0.1286 - categorical_accuracy: 0.150 - ETA: 52s - loss: 0.1305 - categorical_accuracy: 0.156 - ETA: 51s - loss: 0.1317 - categorical_accuracy: 0.160 - ETA: 51s - loss: 0.1326 - categorical_accuracy: 0.140 - ETA: 51s - loss: 0.1310 - categorical_accuracy: 0.142 - ETA: 50s - loss: 0.1327 - categorical_accuracy: 0.150 - ETA: 50s - loss: 0.1328 - categorical_accuracy: 0.147 - ETA: 50s - loss: 0.1328 - categorical_accuracy: 0.151 - ETA: 50s - loss: 0.1336 - categorical_accuracy: 0.151 - ETA: 49s - loss: 0.1354 - categorical_accuracy: 0.147 - ETA: 49s - loss: 0.1343 - categorical_accuracy: 0.147 - ETA: 49s - loss: 0.1347 - categorical_accuracy: 0.140 - ETA: 49s - loss: 0.1347 - categorical_accuracy: 0.145 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.1402 - categorical_accuracy: 0.145 - ETA: 23s - loss: 0.1406 - categorical_accuracy: 0.145 - ETA: 23s - loss: 0.1406 - categorical_accuracy: 0.144 - ETA: 22s - loss: 0.1406 - categorical_accuracy: 0.144 - ETA: 22s - loss: 0.1405 - categorical_accuracy: 0.144 - ETA: 22s - loss: 0.1405 - categorical_accuracy: 0.144 - ETA: 22s - loss: 0.1405 - categorical_accuracy: 0.144 - ETA: 22s - loss: 0.1405 - categorical_accuracy: 0.143 - ETA: 21s - loss: 0.1404 - categorical_accuracy: 0.143 - ETA: 21s - loss: 0.1404 - categorical_accuracy: 0.143 - ETA: 21s - loss: 0.1402 - categorical_accuracy: 0.144 - ETA: 21s - loss: 0.1402 - categorical_accuracy: 0.144 - ETA: 21s - loss: 0.1402 - categorical_accuracy: 0.144 - ETA: 20s - loss: 0.1401 - categorical_accuracy: 0.144 - ETA: 20s - loss: 0.1403 - categorical_accuracy: 0.144 - ETA: 20s - loss: 0.1403 - categorical_accuracy: 0.144 - ETA: 20s - loss: 0.1403 - categorical_accuracy: 0.144 - ETA

In [71]:
filter_length = 300
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam


# model = Sequential()
# model.add(Embedding(6119, 71, input_length=1200))
# model.add(Dropout(0.1))
# model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
# model.add(GlobalMaxPool1D())
# model.add(Dense(82))

model = Sequential()
# model.add(Embedding(max_words, 20, input_length=maxlen))
model.add(Embedding(vocab_size, output_dim=50, input_length=1200))
model.add(Dropout(0.15))
model.add(GlobalMaxPool1D())
model.add(Dense(71, activation='sigmoid'))

# model.summary()


model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 1200, 50)          4939100   
_________________________________________________________________
dropout_12 (Dropout)         (None, 1200, 50)          0         
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 50)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 71)                3621      
_________________________________________________________________
activation_4 (Activation)    (None, 71)                0         
Total params: 4,942,721
Trainable params: 4,942,721
Non-trainable params: 0
_________________________________________________________________
Train on 8540 samples, validate on 949 samples
Epoch 1/5


4672/8540 [===============>..............] - ETA: 8:12 - loss: 0.9538 - categorical_accuracy: 0.0000e+ - ETA: 4:32 - loss: 0.9538 - categorical_accuracy: 0.0312   - ETA: 3:18 - loss: 0.9512 - categorical_accuracy: 0.02 - ETA: 2:40 - loss: 0.9517 - categorical_accuracy: 0.01 - ETA: 2:18 - loss: 0.9517 - categorical_accuracy: 0.01 - ETA: 2:03 - loss: 0.9506 - categorical_accuracy: 0.01 - ETA: 1:52 - loss: 0.9500 - categorical_accuracy: 0.01 - ETA: 1:44 - loss: 0.9495 - categorical_accuracy: 0.01 - ETA: 1:38 - loss: 0.9484 - categorical_accuracy: 0.01 - ETA: 1:33 - loss: 0.9485 - categorical_accuracy: 0.02 - ETA: 1:28 - loss: 0.9484 - categorical_accuracy: 0.02 - ETA: 1:25 - loss: 0.9476 - categorical_accuracy: 0.03 - ETA: 1:22 - loss: 0.9471 - categorical_accuracy: 0.02 - ETA: 1:19 - loss: 0.9464 - categorical_accuracy: 0.02 - ETA: 1:17 - loss: 0.9464 - categorical_accuracy: 0.02 - ETA: 1:15 - loss: 0.9459 - categorical_accuracy: 0.02 - ETA: 1:14 - loss: 0.9456 - categorical_accuracy: 0.

8540/8540 [==============================] - ETA: 25s - loss: 0.8606 - categorical_accuracy: 0.038 - ETA: 25s - loss: 0.8598 - categorical_accuracy: 0.038 - ETA: 24s - loss: 0.8590 - categorical_accuracy: 0.038 - ETA: 24s - loss: 0.8583 - categorical_accuracy: 0.038 - ETA: 24s - loss: 0.8575 - categorical_accuracy: 0.039 - ETA: 24s - loss: 0.8567 - categorical_accuracy: 0.039 - ETA: 23s - loss: 0.8560 - categorical_accuracy: 0.040 - ETA: 23s - loss: 0.8553 - categorical_accuracy: 0.040 - ETA: 23s - loss: 0.8546 - categorical_accuracy: 0.040 - ETA: 23s - loss: 0.8538 - categorical_accuracy: 0.040 - ETA: 23s - loss: 0.8531 - categorical_accuracy: 0.040 - ETA: 22s - loss: 0.8524 - categorical_accuracy: 0.041 - ETA: 22s - loss: 0.8516 - categorical_accuracy: 0.041 - ETA: 22s - loss: 0.8509 - categorical_accuracy: 0.042 - ETA: 22s - loss: 0.8502 - categorical_accuracy: 0.042 - ETA: 21s - loss: 0.8495 - categorical_accuracy: 0.044 - ETA: 21s - loss: 0.8488 - categorical_accuracy: 0.044 - ETA

4672/8540 [===============>..............] - ETA: 55s - loss: 0.7039 - categorical_accuracy: 0.312 - ETA: 52s - loss: 0.7034 - categorical_accuracy: 0.218 - ETA: 52s - loss: 0.7033 - categorical_accuracy: 0.156 - ETA: 51s - loss: 0.7037 - categorical_accuracy: 0.140 - ETA: 51s - loss: 0.7039 - categorical_accuracy: 0.131 - ETA: 51s - loss: 0.7038 - categorical_accuracy: 0.119 - ETA: 51s - loss: 0.7038 - categorical_accuracy: 0.129 - ETA: 51s - loss: 0.7038 - categorical_accuracy: 0.136 - ETA: 51s - loss: 0.7036 - categorical_accuracy: 0.142 - ETA: 51s - loss: 0.7036 - categorical_accuracy: 0.134 - ETA: 50s - loss: 0.7035 - categorical_accuracy: 0.130 - ETA: 50s - loss: 0.7035 - categorical_accuracy: 0.127 - ETA: 50s - loss: 0.7035 - categorical_accuracy: 0.129 - ETA: 50s - loss: 0.7034 - categorical_accuracy: 0.127 - ETA: 49s - loss: 0.7032 - categorical_accuracy: 0.129 - ETA: 49s - loss: 0.7032 - categorical_accuracy: 0.134 - ETA: 49s - loss: 0.7031 - categorical_accuracy: 0.130 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.6992 - categorical_accuracy: 0.126 - ETA: 23s - loss: 0.6992 - categorical_accuracy: 0.127 - ETA: 23s - loss: 0.6992 - categorical_accuracy: 0.127 - ETA: 22s - loss: 0.6992 - categorical_accuracy: 0.128 - ETA: 22s - loss: 0.6991 - categorical_accuracy: 0.128 - ETA: 22s - loss: 0.6991 - categorical_accuracy: 0.128 - ETA: 22s - loss: 0.6991 - categorical_accuracy: 0.128 - ETA: 22s - loss: 0.6991 - categorical_accuracy: 0.128 - ETA: 21s - loss: 0.6991 - categorical_accuracy: 0.128 - ETA: 21s - loss: 0.6990 - categorical_accuracy: 0.128 - ETA: 21s - loss: 0.6990 - categorical_accuracy: 0.128 - ETA: 21s - loss: 0.6990 - categorical_accuracy: 0.128 - ETA: 21s - loss: 0.6990 - categorical_accuracy: 0.127 - ETA: 20s - loss: 0.6990 - categorical_accuracy: 0.128 - ETA: 20s - loss: 0.6990 - categorical_accuracy: 0.128 - ETA: 20s - loss: 0.6989 - categorical_accuracy: 0.128 - ETA: 20s - loss: 0.6989 - categorical_accuracy: 0.129 - ETA

...

8540/8540 [==============================] - ETA: 23s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 23s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 23s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 22s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 22s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 22s - loss: 0.6944 - categorical_accuracy: 0.132 - ETA: 22s - loss: 0.6944 - categorical_accuracy: 0.133 - ETA: 22s - loss: 0.6944 - categorical_accuracy: 0.132 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.132 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.132 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 21s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 20s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 20s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA: 20s - loss: 0.6944 - categorical_accuracy: 0.131 - ETA

4672/8540 [===============>..............] - ETA: 50s - loss: 0.6936 - categorical_accuracy: 0.093 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.109 - ETA: 51s - loss: 0.6935 - categorical_accuracy: 0.083 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.101 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.093 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.099 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.107 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.117 - ETA: 51s - loss: 0.6936 - categorical_accuracy: 0.114 - ETA: 50s - loss: 0.6936 - categorical_accuracy: 0.115 - ETA: 50s - loss: 0.6937 - categorical_accuracy: 0.113 - ETA: 50s - loss: 0.6937 - categorical_accuracy: 0.114 - ETA: 49s - loss: 0.6937 - categorical_accuracy: 0.113 - ETA: 49s - loss: 0.6937 - categorical_accuracy: 0.116 - ETA: 49s - loss: 0.6936 - categorical_accuracy: 0.116 - ETA: 49s - loss: 0.6936 - categorical_accuracy: 0.113 - ETA: 49s - loss: 0.6936 - categorical_accuracy: 0.114 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.6935 - categorical_accuracy: 0.092 - ETA: 23s - loss: 0.6935 - categorical_accuracy: 0.092 - ETA: 23s - loss: 0.6935 - categorical_accuracy: 0.092 - ETA: 22s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 22s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 22s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 22s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 22s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 21s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 21s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 21s - loss: 0.6935 - categorical_accuracy: 0.091 - ETA: 21s - loss: 0.6935 - categorical_accuracy: 0.090 - ETA: 21s - loss: 0.6935 - categorical_accuracy: 0.090 - ETA: 20s - loss: 0.6935 - categorical_accuracy: 0.090 - ETA: 20s - loss: 0.6935 - categorical_accuracy: 0.089 - ETA: 20s - loss: 0.6935 - categorical_accuracy: 0.090 - ETA: 20s - loss: 0.6935 - categorical_accuracy: 0.090 - ETA

4672/8540 [===============>..............] - ETA: 53s - loss: 0.6934 - categorical_accuracy: 0.0000e+0 - ETA: 52s - loss: 0.6934 - categorical_accuracy: 0.0312    - ETA: 51s - loss: 0.6934 - categorical_accuracy: 0.031 - ETA: 52s - loss: 0.6934 - categorical_accuracy: 0.046 - ETA: 51s - loss: 0.6934 - categorical_accuracy: 0.037 - ETA: 51s - loss: 0.6934 - categorical_accuracy: 0.036 - ETA: 51s - loss: 0.6934 - categorical_accuracy: 0.044 - ETA: 51s - loss: 0.6934 - categorical_accuracy: 0.046 - ETA: 50s - loss: 0.6934 - categorical_accuracy: 0.045 - ETA: 50s - loss: 0.6934 - categorical_accuracy: 0.050 - ETA: 50s - loss: 0.6934 - categorical_accuracy: 0.051 - ETA: 50s - loss: 0.6934 - categorical_accuracy: 0.054 - ETA: 49s - loss: 0.6934 - categorical_accuracy: 0.052 - ETA: 49s - loss: 0.6934 - categorical_accuracy: 0.049 - ETA: 49s - loss: 0.6934 - categorical_accuracy: 0.045 - ETA: 49s - loss: 0.6934 - categorical_accuracy: 0.043 - ETA: 48s - loss: 0.6934 - categorical_accuracy: 0.0

8540/8540 [==============================] - ETA: 23s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 23s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 23s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 22s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 22s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 22s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 22s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 22s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 21s - loss: 0.6933 - categorical_accuracy: 0.051 - ETA: 21s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 21s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 21s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 21s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 20s - loss: 0.6933 - categorical_accuracy: 0.050 - ETA: 20s - loss: 0.6933 - categorical_accuracy: 0.049 - ETA: 20s - loss: 0.6933 - categorical_accuracy: 0.049 - ETA: 20s - loss: 0.6933 - categorical_accuracy: 0.049 - ETA

In [72]:
callbacks = [
    ReduceLROnPlateau(),
    EarlyStopping(patience=4),
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]
history = model.fit(padded_docs_train, y_train,
                    class_weight='balanced',
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=callbacks)

Train on 8540 samples, validate on 949 samples
Epoch 1/5


4640/8540 [===============>..............] - ETA: 55s - loss: 0.6933 - categorical_accuracy: 0.062 - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.078 - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.062 - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.085 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.087 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.078 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.075 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.070 - ETA: 52s - loss: 0.6933 - categorical_accuracy: 0.072 - ETA: 52s - loss: 0.6933 - categorical_accuracy: 0.071 - ETA: 52s - loss: 0.6933 - categorical_accuracy: 0.073 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.072 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.067 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.069 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.064 - ETA: 50s - loss: 0.6933 - categorical_accuracy: 0.064 - ETA: 50s - loss: 0.6933 - categorical_accuracy: 0.062 - ETA

8540/8540 [==============================] - ETA: 24s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 24s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA

4672/8540 [===============>..............] - ETA: 55s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.041 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.043 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.043 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.041 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.040 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.040 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.044 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA

...

8540/8540 [==============================] - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.039 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA

4672/8540 [===============>..............] - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.062 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.072 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.078 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.062 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.057 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.058 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.062 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.059 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.056 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.056 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.054 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.050 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.047 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.050 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.049 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.037 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.038 - ETA

4672/8540 [===============>..............] - ETA: 54s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 53s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.020 - ETA: 52s - loss: 0.6932 - categorical_accuracy: 0.031 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.043 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.052 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.049 - ETA: 51s - loss: 0.6932 - categorical_accuracy: 0.050 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.048 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.043 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.042 - ETA: 50s - loss: 0.6932 - categorical_accuracy: 0.049 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.052 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.055 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.054 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.050 - ETA: 49s - loss: 0.6932 - categorical_accuracy: 0.051 - ETA

8540/8540 [==============================] - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 23s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 22s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.046 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 21s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA: 20s - loss: 0.6932 - categorical_accuracy: 0.045 - ETA

In [73]:
# filter_length = 300

# model = Sequential()
# model.add(Embedding(6119, 71, input_length=1200))
# model.add(Dropout(0.5))
# model.add(Conv1D(300, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
# model.add(Conv1D(200, 3, activation='relu'))
# model.add(GlobalMaxPool1D())
# model.add(Dense(82))
# model.add(Activation('sigmoid'))

# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
# model.summary()

In [ ]:
# model.fit(x_train, y_train, batch_size=16, epochs=10)
# log_path=str('C:\Users\Cyborg\Documents\GitHub\Movie-Plot-Synopses')
# model_saver =  ModelCheckpoint(log_path + "/model.ckpt.{epoch:04d}.hdf5", monitor='loss', period=1)
# callbacks = [tensorboard, TerminateOnNaN(), model_saver]


model.fit(padded_docs_train, y_train,
          class_weight='balanced',
          epochs=12,
          batch_size=32,
          validation_split=0.2)
predictions=model.predict(padded_docs_test)

In [ ]:
# score = model.evaluate(x_test, y_test, batch_size=16)

In [ ]:
model = Sequential()
# Configuring the parameters
model.add(LSTM(6119, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()
# pred=[]
# x= model.predict(y_test)
# for i in x:
#     pred.append(i[:])  
    

In [ ]:
# pred[0].shape

In [ ]:
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_test, pred)

In [ ]:
# model.predict(y_test[0]).shape

In [ ]:
# model = Sequential()
# model.add(Embedding(vocab_size, output_dim=50))
# # model.add(LSTM(300))
# # model.add(LSTM(256, return_sequences=True))
# model.add(LSTM(128))
# model.add(Dropout(0.5))
# # model.add(LSTM(128))

# # model.add(LSTM(150))

# # model.add(Dropout(0.5))
# model.add(Dense(71, activation='sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.summary()

In [ ]:
# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])


In [ ]:
# model.fit(padded_docs_train, y_train,
#           class_weight='balanced',
#           epochs=12,
#           batch_size=32,
#           validation_split=0.2)